# Time Travel Debugger project

## Personal Information

In [4]:
PROJECT_TYPE = 2
NAME = ["Daniel Gusenburger", "Daniel Tabellion"]
ID = ["2544941", "2555742"]
IMPLEMENTED = set()

## High-level Explanation:

We structured our code into `model`, `view` and `domain` folders.

Our debugger starts by running through the given function that should be examined, tracing every execution point.
The `TimeTravelTracer` in the `tracer.py` module is responsible for building up the information we need for later stepping through the given function. Each point of execution is captured by an instance of an `ExecStateDiff`.
Each `ExecStateDiff` stores a list of currently open function scopes(represented by the model class `FunctionStateDiff`) and it's action, that it performed("CALL","RETURN", "UPDATE" and "EXCEPTION"). These function scopes contain some information about the function itself, added variables and the values of changed variable, before and after the update.
The tracer also builds a source map, that contains all source lines of functions that we trace.

When done tracing, the list of diffs is given to the `TimeTravelDebugger` in the `debugger.py`.
This class is responsible for taking commands and mapping these to simple step commands, implemented in the `StateMachine`. 
The `StateMachine` keeps track of the absolute values for variables and class members for open function scopes while stepping through the programm.

With this compartmenalization we make sure, that we can use the main debugger implementation for both the CLI and the GUI.

## Features:


**/R1 `Quit`:**
    
For quitting we just call sys.exit(1).
Also we catch KeyboardInterrupt and EOF exceptions, such that CTRL-C does not leave the debugger, which leads to a cleaner user experience.

One can leave the debugger via the `quit` command or with CTRL-D, which opens a prompt, that asks if you really want to leave.


In [5]:
IMPLEMENTED.add("/R1/")

**/R2 `Help`:**

For the help command, we used the already present structure from the lecture, so every command, that is named `*_command` is interpreted as a command and appears in the help list. Also docstrings are printed after each command, to give a better understanding of the commands.

In [6]:
IMPLEMENTED.add("/R2/")

**/R3:**

For each command we performed some argument validation in the corresponding `debugger.py` commands.

In [13]:
IMPLEMENTED.add("/R3/")

**/R4 `Step` and /R5 `Backstep`:**

For stepping forwards (and backwards) we implemented the `StateMachine` class which, as the name implies, represents a simple state machine (We can interpret all possible states of the program to be debugged as a simple state machine, where our stored diffs are translations and the absolute current state of variables in function scopes are the nodes).
The purpose of the state machine is to implement `step` and `backstep`, in order to later build more complex movement commands leveraging these two.

Importantly, for this we need `step` and `backstep` to correctly build(or restore) the absolute state of variables from the given diffs.
In order to make this easier we added the `FunctionStates` helper class, which is a dictionary that maps from function names to a list of open scopes and their local variable values at the current point in time. Also for each function we store, which of these scopes is the active one. This represents the absolute state of any function scope at any given point in time.
The StateMachine always stores its current point in the diff list, so it knows what action comes next.

When calling a function, we append a new scope (dictionary of variables) to the list of open scopes for this function, add the parameters given by the current diff to that dictionary and point at that new scope as active.
Step one diff further.

Reverting a function call deletes the topmost scope of the called function(We don't lose information here, since we can restore its state again, when performing the call while stepping forward).
Go one diff back.

When we return from a function we do nothing but go to the next diff.
We do not delete the newest scope, since we want to keep it in case we step backwards and need to restore it.
We also don't need to update or add any variables, since a return always triggers after we updated the state for the last line of a function.

Reverting a return does nothing but going one diff back, for the same reasons as for return.

When not doing a call or a return, we update the state of the variables, given the next diff.
We add any added variables and update the state of variables that got changed in the current scope of the active function.

Reverting an update removes added variables from the active scope and reverts the values before the update(stored in the diff).

If the current diff's action is an "EXCEPTION" we do nothing, since this is the end of the tracing and printing the exception is part of the CLI, not the debugger class.

By showing that these two actions are sound we can make our lifes a lot easier for all other movement commands!

We also have a `@trigger_update` annotation for each movement function that triggers the UI to update. This ensures, that we always print the current state of the pointer in the code.

In [7]:
IMPLEMENTED.add("/R4/")

In [8]:
IMPLEMENTED.add("/R5/")

**/R6 `next` and /R7 `previous`:**

`next` is the same as `until` without parameters.
`previous` is the same as `backuntil` without parameters.
See until/backuntil.

In [9]:
IMPLEMENTED.add("/R6/")

In [10]:
IMPLEMENTED.add("/R7/")

**/R8 `finish`:**

Call `step` until the current diff's stored action is "return" and the current diff is in the same "depth" of execution.
With depth we denote the number of open functions (like in a callstack).
By that we ensure, that we ignore return actions from other functions and we realy only run right before the return action of the current function is performed.

If we perform `finish` on the end of a function this condition obviously holds and we only update the UI.

In [11]:
IMPLEMENTED.add("/R8/")

**/R9 `start`:**

Similar as for `finish`, but call `backstep` until the current diff's  action in the same depth is "CALL".

In [13]:
IMPLEMENTED.add("/R9/")

**/R10/ `until`:**

For the until command we wrote a argument parser function in the `debugger.py`, that maps the given arguments to just a line number and optionally the filename, which makes writing `until` easier.
This is done via the source map, that we build during parsing.
We can map the current functions name, given by the current diff, to the full source code lines of this function.

**/R100 `until \<line_number\>`:**

We implement this like the other requirements for until, so that we search for the next occurence of the given line (or next executable line, skipping comments etc.), iterating through loops if neccesary (this is fine according to the [forum](https://cms.cispa.saarland/debug/forum/viewtopic.php?f=10&t=44)).
When no line is given act like `next`. So step to next (executable) line and if at the end of a function step out of it.
This is why the `next` command is just a specialization of `until` and we can implement it as `until` without parameters.

So to implement this feature we first determine our target line, we want to step to. When a line number is given, this is the target, if not the target is the current line + 1 (move to the next line). Then we check if this target is actually an executable line, adjusting it to the next possible executable line (For this we essentially loop over the source of the current function and for each line we check if it contains comments etc).
Then when this target is determined, `step` until we either hit the target or we return from a function (stepping out of the function and staying there).

**/R101 `until \<filename\>:\<line_number\>`:**

This works the same as in /R100, but we additionally check whether the target line is hit in a specific file (we store the file name of a given line in the diff as well).

**/R102 `until \<function_name\>`:**

The parser converts function names to the corresponding line number, so this is the same as R/100.

**/R103 `until \<filename\>:\<function_name\>`:**

The parser converts function names to the corresponding line number, so this is the same as R/101.

**/R110-R113/:**

Analogous to /R100-R103/, but step backwards and check if a function got called.

In [14]:
print(IMPLEMENTED)

{'/R3/', '/R7/', '/R9/', '/R8/', '/R1/', '/R2/', '/R4/', '/R6/', '/R5/'}


## Implementation

In [ ]:
from time_travel_debugger.view.gui import GUI
from time_travel_debugger.view.cli import TimeTravelCLI
from main import remove_html_markup

    
with GUI():
    remove_html_markup("<tag>hallo</tag>")

## Presentation

## Summary